In [ ]:
%reset -f
%load_ext autoreload
%autoreload

In [ ]:
import numpy as np
import crc
import tensorflow as tf
import keras
from sklearn import preprocessing
import pickle
import time

In [ ]:
scenario = 'NS'
n = 3
# modes = ['2,2', '2,1', '3,3', '3,2', '4,4', '4,3']
modes = ['2,2', '4,3', '4,4']

In [ ]:
# time_indexes, B  = {'A': {}, 'phi': {}}, {'A': {}, 'phi': {}}
# models, yscalers = {'A': {}, 'phi': {}}, {'A': {}, 'phi': {}}
# for mode in modes:
#     for component in time_indexes:
#         _, time_indexes[component][mode], B[component][mode] = crc.load_bits('NS', mode, n, component)
#         with open(f"../models/scaler{scenario}{n}_{mode.replace(',', '_')}_{component}.pickle", 'rb') as infile:
#             yscaler_comp = pickle.load(infile)
#             yscalers[component][mode] = infile
#         models[component][mode] = tf.keras.saving.load_model(f"../models/model{scenario}{n}_{mode.replace(',', '_')}_{component}.keras")

In [1]:
def SEOBNRv5_CRCNS(params: np.ndarray, modes: list = ['2,2', '2,1', '3,3', '3,2', '4,4', '4,3']) -> np.ndarray:

    # loading bits 
    TIME = crc.GET_TIME()
    time_indexes, B = {'A': {}, 'phi': {}}, {'A': {}, 'phi': {}}
    ANNs, scalers   = {'A': {}, 'phi': {}}, {'A': {}, 'phi': {}}
    for mode in modes:
        for component in time_indexes:
            _, time_indexes[component][mode], B[component][mode] = crc.load_bits('NS', mode, n, component)
            with open(f"../models/scaler{scenario}{n}_{mode.replace(',', '_')}_{component}.pickle", 'rb') as infile:
                scaler_comp = pickle.load(infile)
                scalers[component][mode] = scaler_comp
            ANNs[component][mode] = tf.keras.saving.load_model(f"../models/model{scenario}{n}_{mode.replace(',', '_')}_{component}.keras")

    # want to time each mode generation
    waveforms, stats = {}, {}
    for mode in modes:
        # t_i = time.time()
        predict_A   = scalers['A'][mode].inverse_transform(ANNs['A'][mode].predict(params))
        predict_phi = scalers['phi'][mode].inverse_transform(ANNs['phi'][mode].predict(params))

        surrogate_timing = []
        reconstructed_full = np.zeros((len(params), len(TIME)), dtype=np.complex128)
        for i, q in zip([i for i in range(len(params))], [float(q) for q in params]):
            t_i = time.time()
            Ih_A                   = crc.EI_h(predict_A[i], B['A'][mode], time_indexes['A'][mode])
            Ih_phi                 = crc.EI_h(predict_phi[i], B['phi'][mode], time_indexes['phi'][mode])        
            reconstructed_full[i] += Ih_A * np.exp(-1j * Ih_phi)
            surrogate_timing.append(time.time() - t_i)
        waveforms[mode], stats[mode] = reconstructed_full, np.array(surrogate_timing)
        
    return waveforms, stats

NameError: name 'np' is not defined

In [ ]:
# define params
params = np.linspace(1, 4, 1000)[:, np.newaxis]

In [ ]:
wf, st = SEOBNRv5_CRCNS(params, modes)

In [ ]:
st